# Support Vector Machine
#### Ruixuan Dong
---

### Table of Contents

 - [The Support Vector Machine classification algorithm](#1)
    - [Optimal separating hyperplane](#11)
    - [Algorithm Explanation](#12)
 - [Research Problem -- Using synthetic data to classify different systolic levels](#2)
    - [Overview of the Problem set](#21)
    - [Implement SVM on Synthetic data](#22)
 
 

<a name='1'></a>
### 1 - The Support Vector Machine classification algorithm

First, let's run the cell below to import all the packages that we will need during this tutorial.

In [8]:
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

<a name='11'></a>
### 1-1 Optimal separating hyperplane


A refinement of the perceptron learning algorithm is the optimal separating hyperplane. The goal is to find a hyperplane which separates the two classes and maximizes the distance from the hyperplane to the closest point from either class. In so doing, this leads to a unique solution to separating hyperplane problem, and tends to lead to better prediction accuracy on new data points.

The optimization problem for finding an optimal separating hyperplane is

\begin{align*}
\max_{\beta, \beta_0, ||\beta||_2=1} M \quad \text{subject to} \quad y_i(x_i'\beta + \beta_0) \geq M, \quad 1 \leq i \leq n. \quad \quad (1)
\end{align*}

Notice, the requirement that $y_i(x_i'\beta + \beta_0) \geq M$ will ensure that all points are correctly classified and the distance from each point to the separating hyperplane is at least $M$. We can rewrite this in a more familiar way by removing the unit length constraint on $\beta$ by letting $\alpha = \beta ||\beta||_2$ and $\alpha_0 = \beta_0 ||\beta||_2$, so that the constraint can be written as $y_i(x_i'\alpha+\alpha_0)/||\alpha||_2 \geq M$, $1 \leq i \leq n$. Hence, we see that an equivalent formulation is

\begin{align*}
\max_{\beta, \beta_0} M \quad \text{subject to} \quad y_i(x_i'\beta + \beta_0) \geq M||\beta||_2, \quad 1 \leq i \leq n.
\end{align*}

For any $\beta, \beta_0$ satisfying $y_i(x_i'\beta + \beta_0) \geq M||\beta||_2$, any scalar multiple of $\beta, \beta_0$ also satisfies the constraint. Thus, we can simply set $M = \frac{1}{||\beta||_2}$, so that we may (finally) write the problem as

\begin{align*}
\min_{\beta, \beta_0} \frac{1}{2}||\beta||_2^2 \quad \text{subject to} \quad y_i(x_i'\beta + \beta_0) \geq 1, \quad 1 \leq i \leq n. \quad \quad (2)
\end{align*}

From this formulation, we can see that the optimal separating hyperplane is that which falls between two parallel hyperplanes between which no points live (although these hyperplanes may pass through points). The distance from these two hyperplanes and the separating hyperplane is $\frac{1}{||\beta||_2}$ and our objective is to maximize this distance. This is illustrated in Figure 1 below. These hyperplanes are shown in blue, with the orange hyperplane being the optimal separating hyperplane. In Figure 5, this is the distance from the circled points to the orange hyperplane.

Note that the dual problem (an equivalent formulation) to the above is given by

\begin{align*}
\max_\gamma \left( \min_{\beta, \beta_0} \left( \frac{1}{2}||\beta||_2^2 + \sum_{i=1}^{n} \gamma_i [y_i(x_i'\beta + \beta_0) - 1] \right) \right) \quad \text{subject to} \quad \gamma_i \geq 0.
\end{align*}


where the $\gamma_i$ are the dual variables (i.e., lagrangian multipliers).

<img src="hyperplane.png" alt="Figure 1" width="400" height="300">
<p style="text-align:left;">Figure 1: Plot showing data which can be separated by a hyperplane, but linear regression
on $y$ (solid black) yields a decision boundary which does not separate the data. The optimal separating hyperplane is given in orange, with the support points circled in blue.</p>

<a name='12'></a>
### 1-2 Algorithm Explanation

Support vector machines can be thought of as a generalization of the optimal separating hyperplane. For the proceeding discussion, let us define the space between the two hyperplanes which are determined by the support points as the margin. Based on the original formulation of the optimal separating hyperplane, we can think of the margin as having a width equal to $2\hat{M}$, where $\hat{M}$ is the solution to (1).

Suppose now that the data are not separable by a hyperplane. Rather than requiring 

\begin{align*}
y_i(x_i'\beta + \beta_0) \geq M,
\end{align*}

as in the optimal separating hyperplane, we will relax this restriction to require 

\begin{align*}
y_i(x_i'\beta + \beta_0) \geq M(1 - \xi_i) = M - M\xi_i.
\end{align*}

where $\xi_1, \ldots, \xi_n$ are slack variables such that
$\xi_i \geq 0$,
$\sum_{i=1}^{n} \xi_i \leq \text{constant}$.

Intuitively, this is straightforward to interpret – the value of $\xi_i$ is the proportional amount by which the prediction $x_i'\beta + \beta_0$ is on the incorrect side of the margin (relative to the hyperplanes which determine the margin). Misclassifications occur when $\xi_i > 1$. Thus, a bound on $\sum_{i=1}^{n} \xi_i$ controls the total distance from all misclassified points to the correct side of the margin. Bounding this sum at $L$ bounds the number of training misclassifications at $L$. This is illustrated in Figure 2.

Following the reformulation of (2) into (3), we can write the corresponding optimization problem as

\begin{align*}
\min_{\beta, \beta_0} \frac{1}{2}||\beta||_2^2 \quad \text{subject to} \quad 
\begin{cases}
    y_i(x_i'\beta + \beta_0) \geq 1 - \xi_i & \text{for } 1 \leq i \leq n \\
    \xi_i \geq 0 & \text{for } 1 \leq i \leq n \\
    \sum_{i=1}^{n} \xi_i \leq \text{constant}
\end{cases}
\end{align*}

This optimization is often presented in its dual form

\begin{align*}
\max_{\gamma, \mu} \left( \min_{\beta, \beta_0, \xi} \left( \frac{1}{2}||\beta||_2^2 + L \sum_{i=1}^{n} \xi_i - \sum_{i=1}^{n} \gamma_i [y_i(x_i'\beta + \beta_0) - (1 - \xi_i)] - \sum_{i=1}^{n} \mu_i \xi_i \right) \right)
\end{align*}

subject to $\mu_i \geq 0$, $\xi_i \geq 0$, $\gamma_i \geq 0$ for $1 \leq i \leq n$.

where $L$ is some constant corresponding to the constraint on $\sum_{i=1}^{n} \xi_i$
from before.

<img src="SVM.png" alt="Figure 2" width="600" height="300">
<p style="text-align:left;">Figure 2: A visualization of the slack variables in support vector machines. The points labeled $\xi^*_j$ are on the wrong side of their margin by an amount $\xi^*_j = M\xi_j$. Points on the correct side have $\xi^*_j = 0$. The margin is maximized subject to a total budget of $\sum_{j=1}^{n} \xi_j \leq$ a constant.</p>


<a name='2'></a>
### 2 - Using synthetic data to classify different systolic levels

In this part, we use the dataset generated by Synthetic SCG data generator https://github.com/wsonguga/DataDemo. In the data file, each row includes sensor data (10 seconds * 100Hz) + HeartRate + RespiratoryRate + SystolicBloodPressure + DiastolicBloodPressure.

Now we would like to give a introduction about the synthetic SCG dataset we generated. 

Based on Synthetic SCG data generator, we generated an artificial (synthetic) scg signal of a given duration (10 seconds, i.e. duration=10) and sampling rate (100Hz, i.e. sampling rate=100) using a model based on Daubechies wavelets to roughly approximate cardiac cycles.

Besides, we set 
 - heart rate to be randomly chosen from the intgers range from 50 to 150, with the desired heart rate standard deviation (beats per minute) equal to 1.
 - respiratory rate to be randomly chosen from the intgers range from 10 to 30
 - diastolic blood pressure to be randomly chosen from the intgers range from 60 to 99
 - systolic blood pressure to be randomly chosen from the intgers range from 100 to 160

The sample size of the current dataset is 6,000 in total.

<a name='21'></a>
### 2-1 Overview of the Problem set 
**Problem Statement:** The generated dataset containing: 
- a dataset set ("total_large.csv") of 6,000 samples labeled as lower (100<=systolic blood pressure<140) or higher (140<=systolic blood pressure<=160) 
- each sample is of shape (1, 1003) where 1003 is for the 1000-d signal and heart rate, respiratory rate and diastolic blood pressure

In this part, we will build a simple SVM classifier that can correctly classify samples as lower or higher (SBP).

Let's get more familiar with the dataset. Load the data by running the following code.

In [2]:
column_names = [str(i) for i in range(1, 1001)] + ['heart_rate', 'respiratory_rate', 'systolic', 'diastolic']
total = pd.read_csv('total.csv', 
                 header=None, 
                 names=column_names)
total.head(3)

,1,2,3,4,5,6,7,8,9,10,...,995,996,997,998,999,1000,heart_rate,respiratory_rate,systolic,diastolic
0,3.993225e-08,3.517546e-07,3.597235e-06,0.000006,-0.000003,0.000001,-2.597618e-07,1.085503e-07,-2.178073e-07,-6.006744e-08,...,-3.069616e-08,-2.657577e-08,-2.138259e-08,-1.922973e-08,-2.095749e-08,-2.288238e-08,124.0,17.0,103.0,68.0
1,3.925762e-08,5.413484e-07,4.186926e-06,0.000007,-0.000002,-0.000002,2.296199e-06,-1.955847e-06,1.334962e-06,-3.836647e-07,...,-2.732173e-08,-2.730453e-08,-2.924664e-08,-2.703059e-08,-2.099285e-08,-1.787339e-08,124.0,30.0,102.0,91.0
2,2.884543e-08,6.735418e-08,9.212940e-07,0.000005,0.000004,-0.000004,2.197260e-06,-1.165844e-06,8.744602e-07,-6.691456e-07,...,-2.810176e-08,-2.713190e-08,-2.622019e-08,-2.368465e-08,-2.027036e-08,-1.836652e-08,121.0,10.0,156.0,92.0


In [3]:
def signal2matrix(total):
    total = total.values

    numberOfLines = len(total)
    returnMat = np.zeros((numberOfLines, 1003))
    classLabelVector = []
    index = 0

    for line in total:
        returnMat[index, :1002] = line[:1002]
        returnMat[index, 1002] = line[1003]
        if 100 <=line[1002]< 140:
            classLabelVector.append(1)
        elif 140 <=line[1002]<= 160:
            classLabelVector.append(2)
        index += 1
    return returnMat, classLabelVector

def autoNorm(dataSet):
    minVals = dataSet.min(0)
    maxVals = dataSet.max(0)
    ranges = maxVals - minVals
    normDataSet = np.zeros(np.shape(dataSet))
    m = dataSet.shape[0]
    normDataSet = dataSet - np.tile(minVals, (m, 1))
    normDataSet = normDataSet / np.tile(ranges, (m, 1))
    return normDataSet, ranges, minVals

Overall, these functions are used for preprocessing data. `signal2matrix` converts data into a matrix format with class labels, and `autoNorm` performs feature scaling to normalize the dataset. These preprocessing steps are often important before applying machine learning algorithms to data.

<a name='22'></a>
### 2-2 Implement SVM on Synthetic data

In [9]:
X, y = signal2matrix(total)
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an SVM classifier
clf = svm.SVC(kernel='linear')  # You can choose different kernels (linear, rbf, etc.)

# Train the SVM classifier on the training data
clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 64.33%


This code demonstrates the process of preparing data, splitting it into training and testing sets, training an SVM classifier, making predictions, and evaluating the classifier's performance using accuracy. The reported accuracy indicates how well the SVM classifier is able to classify the test data based on the provided features. In a binary classification context(our e), this means that about 64.33% of the test data points were assigned the correct class label by the classifier.